In [1]:
!pip install PyPDF2


     ---------------------------------------- 0.0/232.6 kB ? eta -:--:--
     ----- --------------------------------- 30.7/232.6 kB 1.3 MB/s eta 0:00:01
     --------- --------------------------- 61.4/232.6 kB 656.4 kB/s eta 0:00:01
     --------------- -------------------- 102.4/232.6 kB 837.8 kB/s eta 0:00:01
     --------------------------------- ---- 204.8/232.6 kB 1.1 MB/s eta 0:00:01
     -------------------------------------- 232.6/232.6 kB 1.2 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
!git add .
!git commit -m "Update"
!git push

On branch main
Your branch is ahead of 'origin/main' by 2 commits.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean


To https://github.com/Thuylt185411/web_lor_tracy.git
 ! [rejected]        main -> main (fetch first)
error: failed to push some refs to 'https://github.com/Thuylt185411/web_lor_tracy.git'
hint: Updates were rejected because the remote contains work that you do not
hint: have locally. This is usually caused by another repository pushing to
hint: the same ref. If you want to integrate the remote changes, use
hint: 'git pull' before pushing again.
hint: See the 'Note about fast-forwards' in 'git push --help' for details.
